# 64-Band Real-Time Graphical Equalizer
This notebook records audio from the microphone, applies a 64-band graphical equalizer in real-time, and plays it back using sounddevice.

In [1]:
!pip install sounddevice numpy scipy ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.9/914.9 kB 20.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipywidgets]


In [2]:
import numpy as np
import sounddevice as sd
from scipy.signal import butter, sosfilt
import ipywidgets as widgets
from IPython.display import display
import threading

# Audio parameters
fs = 44100
blocksize = 1024
channels = 1

# Frequency bands (log spaced)
freqs = np.logspace(np.log10(20), np.log10(20000), 65)

# Create sliders
sliders = []
for i in range(64):
    slider = widgets.FloatSlider(
        value=0.0,
        min=-12.0,
        max=12.0,
        step=0.5,
        description=f'{int(freqs[i])}Hz',
        orientation='vertical',
        layout=widgets.Layout(height='200px', width='30px')
    )
    sliders.append(slider)

display(widgets.HBox(sliders))

# Design band filters
sos_filters = []
for i in range(64):
    low = freqs[i] / (fs/2)
    high = freqs[i+1] / (fs/2)
    sos = butter(2, [low, high], btype='band', output='sos')
    sos_filters.append(sos)

# Filter states
states = [np.zeros((sos.shape[0], 2)) for sos in sos_filters]

def audio_callback(indata, outdata, frames, time, status):
    global states
    x = indata[:, 0]
    y = np.zeros_like(x)
    
    for i in range(64):
        gain = 10**(sliders[i].value / 20)
        filtered = sosfilt(sos_filters[i], x, zi=states[i])[0]
        y += gain * filtered
    
    outdata[:, 0] = y

stream = sd.Stream(
    samplerate=fs,
    blocksize=blocksize,
    channels=channels,
    callback=audio_callback
)

def start_audio():
    stream.start()

def stop_audio():
    stream.stop()

start_button = widgets.Button(description='Start')
stop_button = widgets.Button(description='Stop')

start_button.on_click(lambda x: start_audio())
stop_button.on_click(lambda x: stop_audio())

display(widgets.HBox([start_button, stop_button]))

In [3]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048
NUM_BANDS = 64

# Global gains array (1.0 = 0dB)
# We use a numpy array so the audio callback can read it efficiently
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    # Perform FFT
    spectrum = rfft(indata[:, 0])
    
    # Map the 64 gain bands to the FFT bins
    # We use linear interpolation for speed
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    # Apply gains and invert back to time domain
    processed_spectrum = spectrum * applied_gains
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic ---
sliders = []
def update_gain(change):
    # 'owner' is the slider that triggered the change
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

# Create 64 vertical sliders
for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0,
        min=0.0,
        max=3.0,  # Max 3x boost
        step=0.1,
        orientation='vertical',
        readout=False,
        layout=widgets.Layout(width='15px', height='200px')
    )
    # Store the index in a custom attribute to identify it in the callback
    slider.description_int = i 
    slider.observe(update_gain, names='value')
    sliders.append(slider)

ui = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll'))

# --- 3. Run the Stream ---
print("Adjust the sliders to change the EQ in real-time.")
display(ui)

try:
    with sd.Stream(samplerate=SAMPLE_RATE,
                   blocksize=BLOCK_SIZE,
                   channels=1,
                   callback=audio_callback):
        print("Audio Stream Active. Press 'Stop' in Jupyter to end.")
        # Keeps the cell running while the background thread handles audio
        while True:
            sd.sleep(1000)
except KeyboardInterrupt:
    print("\nStream stopped.")

Adjust the sliders to change the EQ in real-time.


Audio Stream Active. Press 'Stop' in Jupyter to end.

Stream stopped.


In [4]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048
NUM_BANDS = 64
global_stream = None  # To hold the stream object

# Global gains array (1.0 = 0dB)
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    spectrum = rfft(indata[:, 0])
    
    # Map gains to frequency bins
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    processed_spectrum = spectrum * applied_gains
    
    # Inverse FFT to time domain
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic ---
sliders = []
def update_gain(change):
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='12px', height='150px')
    )
    slider.description_int = i
    slider.observe(update_gain, names='value')
    sliders.append(slider)

# Group sliders
eq_box = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll', width='100%'))

# Create a Stop Button
stop_btn = widgets.Button(
    description='Stop Audio',
    button_style='danger', # Red color
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Audio Stream Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 3. Start the Stream ---
# We use .start() instead of a 'with' block so the cell can finish execution
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)

_IncompleteInputError: incomplete input (3145588784.py, line 78)

In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048
NUM_BANDS = 64
global_stream = None  # To hold the stream object

# Global gains array (1.0 = 0dB)
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    spectrum = rfft(indata[:, 0])
    
    # Map gains to frequency bins
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    processed_spectrum = spectrum * applied_gains
    
    # Inverse FFT to time domain
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic ---
sliders = []
def update_gain(change):
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='12px', height='150px')
    )
    slider.description_int = i
    slider.observe(update_gain, names='value')
    sliders.append(slider)

# Group sliders
eq_box = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll', width='100%'))

# Create a Stop Button
stop_btn = widgets.Button(
    description='Stop Audio',
    button_style='danger', # Red color
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Audio Stream Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 3. Start the Stream ---
# We use .start() instead of a 'with' block so the cell can finish execution
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)
    global_stream.start()
    
    print("Audio is live! Adjust sliders below.")
    display(stop_btn)
    display(eq_box)
    
except Exception as e:
    print(f"Error starting stream: {e}")

Audio is live! Adjust sliders below.


Button(button_style='danger', description='Stop Audio', icon='square', style=ButtonStyle())

In [1]:
import ipywidgets as w; w.IntSlider()

IntSlider(value=0)

In [2]:
w.IntSlider()

IntSlider(value=0)

In [3]:
a = w.IntSlider()

In [4]:
a

IntSlider(value=0)

In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
# Wavelets need larger blocks to avoid edge artifacts and overhead
# 4096 is safer for real-time Python performance with WPD
BLOCK_SIZE = 4096  
NUM_BANDS = 64
WAVELET_NAME = 'db5'  # 'db1' (Haar) is fastest. 'db4' is smoother but slower.

global_stream = None 
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Wavelet Audio Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    # 1. Flatten input to 1D array for PyWavelets
    signal = indata[:, 0]
    
    # 2. Create Wavelet Packet Object
    # 'symmetric' padding reduces edge artifacts slightly
    wp = pywt.WaveletPacket(data=signal, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
    
    # 3. Get the 64 leaf nodes ordered by FREQUENCY
    # (Natural wavelet order is not frequency ordered)
    nodes = wp.get_level(6, order='freq')
    
    # 4. Apply Gains
    # We iterate through our 64 sliders and multiply the node data
    # We use min() to ensure we don't crash if node count differs slightly (rare)
    for i in range(min(len(nodes), len(gains))):
        nodes[i].data = nodes[i].data * gains[i]
        
    # 5. Reconstruct
    # update=False prevents re-computing coefficients we just modified
    new_signal = wp.reconstruct(update=False)
    
    # 6. Safety Check & Reshape
    # Reconstruction might produce slightly different length due to padding
    if len(new_signal) != len(signal):
        # Truncate or pad if necessary (usually just a 1-sample diff)
        new_signal = new_signal[:len(signal)]
        
    outdata[:, 0] = new_signal

# --- 2. The UI Logic (Same as before) ---
sliders = []
def update_gain(change):
    band_index = change['owner'].description_int
    gains[band_index] = change['new']

for i in range(NUM_BANDS):
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='12px', height='150px')
    )
    slider.description_int = i
    slider.observe(update_gain, names='value')
    sliders.append(slider)

eq_box = widgets.HBox(sliders, layout=widgets.Layout(overflow='x scroll', width='100%'))

stop_btn = widgets.Button(
    description='Stop WPD Audio',
    button_style='danger',
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Wavelet Stream Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 3. Start the Stream ---
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    print(f"Starting Wavelet Stream ({WAVELET_NAME}, {NUM_BANDS} bands)...")
    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)
    global_stream.start()
    
    print("Audio is live! Processing with Wavelet Packets.")
    display(stop_btn)
    display(eq_box)
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Wavelet Stream (db5, 64 bands)...
Audio is live! Processing with Wavelet Packets.


Button(button_style='danger', description='Stop WPD Audio', icon='square', style=ButtonStyle())

In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
from scipy.fft import rfft, irfft, rfftfreq
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 2048 
NUM_BANDS = 64
global_stream = None 

# Global gains array
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. Audio Callback (FFT Method) ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    spectrum = rfft(indata[:, 0])
    
    # Map gains to frequency bins (Linear interpolation)
    bin_indices = np.linspace(0, NUM_BANDS - 1, len(spectrum)).astype(int)
    applied_gains = gains[bin_indices]
    
    processed_spectrum = spectrum * applied_gains
    outdata[:, 0] = irfft(processed_spectrum, n=len(indata))

# --- 2. The UI Logic with Labels ---
slider_containers = []

def update_gain(change):
    # 'owner' is the slider object
    new_val = change['new']
    # We stored the index and the label widget inside the slider object for easy access
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    
    # Update global gain array
    gains[band_index] = new_val
    
    # Update the label text
    label_widget.value = f"{new_val:.1f}"

for i in range(NUM_BANDS):
    # Create the Label
    lbl = widgets.Label(
        value="1.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    
    # Create the Slider
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    
    # Link them together
    slider.band_index = i
    slider.label_ref = lbl  # Store reference to label inside slider
    slider.observe(update_gain, names='value')
    
    # Stack them vertically (Label on top, Slider below)
    # We use VBox to group them
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

# Group all 64 columns into one scrollable row
eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Audio',
    button_style='danger',
    icon='square'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
    stop_btn.disabled = True
    print("Stream Stopped.")

stop_btn.on_click(stop_stream)

# --- 4. Start Stream ---
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.Stream(samplerate=SAMPLE_RATE,
                              blocksize=BLOCK_SIZE,
                              channels=1,
                              callback=audio_callback)
    global_stream.start()
    
    print("Audio Live. Gains displayed above sliders.")
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error: {e}")

Audio Live. Gains displayed above sliders.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096  # Larger block size for stable Wavelet processing
NUM_BANDS = 64
WAVELET_NAME = 'db1' # 'db1' is fast. Try 'db4' for smoother filtering if CPU allows.
NOISE_VOLUME = 0.2   # Safety volume (White noise is loud!)

global_stream = None 

# Global gains array (Start at 1.0)
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The White Noise + Wavelet Callback ---
def audio_callback(indata, outdata, frames, time, status):
    if status:
        print(status)
    
    # A. Generate White Noise (Uniform distribution between -1 and 1)
    # We ignore 'indata' entirely.
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    # B. Wavelet Decomposition
    try:
        # Create Wavelet Packet tree
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # C. Apply Gains
        # Ensure we don't exceed bounds if node count varies slightly
        count = min(len(nodes), len(gains))
        for i in range(count):
            # Multiply the coefficients of this band by the slider value
            nodes[i].data = nodes[i].data * gains[i]
            
        # D. Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # E. Safety Clipping & Buffer Filling
        # Wavelet reconstruction length can vary by a few samples due to padding
        # We truncate or pad to match the requested 'frames' size
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        # Write to output (Mono)
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic (Sliders + Labels) ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    
    # Update global array
    gains[band_index] = new_val
    # Update label text
    label_widget.value = f"{new_val:.1f}"

# Create 64 Sliders with Labels
for i in range(NUM_BANDS):
    # Label
    lbl = widgets.Label(
        value="1.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=1.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    
    # Linkage
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    # Stack vertically
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

# Scrollable Container
eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting White Noise Generator with {NUM_BANDS}-band Wavelet EQ...")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    # We use an OutputStream since we are generating data, not capturing it.
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting White Noise Generator with 64-band Wavelet EQ...


Exception ignored from cffi callback <function _StreamBase.__init__.<locals>.callback_ptr at 0x7f3efc61aa30>:
Traceback (most recent call last):
  File "/home/vruiz/envs/intercom/lib/python3.14/site-packages/sounddevice.py", line 878, in callback_ptr
    return _wrap_callback(callback, data, frames, time, status)
  File "/home/vruiz/envs/intercom/lib/python3.14/site-packages/sounddevice.py", line 2777, in _wrap_callback
    callback(*args)
TypeError: audio_callback() missing 1 required positional argument: 'status'


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096  # Larger block size for stable Wavelet processing
NUM_BANDS = 64
WAVELET_NAME = 'db1' # 'db1' is fast.
NOISE_VOLUME = 0.1   # Safety volume

global_stream = None 
gains = np.ones(NUM_BANDS, dtype=np.float32)

# --- 1. The Corrected Callback (No 'indata') ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Generate White Noise (Uniform distribution)
    # We generate exactly 'frames' worth of noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        # Wavelet reconstruction length can vary by a few samples due to padding
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        # Write to output (Mono)
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic (Same as before) ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.1f}"

for i in range(NUM_BANDS):
    lbl = widgets.Label(
        value="1.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting White Noise Generator with {NUM_BANDS}-band Wavelet EQ...")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    # Using OutputStream (Out Only)
    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting White Noise Generator with 64-band Wavelet EQ...


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# --- CHANGE 1: Initialize gains to 0.0 ---
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.1f}"

for i in range(NUM_BANDS):
    # --- CHANGE 2: Label starts at "0.0" ---
    lbl = widgets.Label(
        value="0.0",
        layout=widgets.Layout(width='20px', justify_content='center')
    )
    
    # --- CHANGE 3: Slider starts at 0.0 ---
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.1,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='150px')
    )
    
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting Silent Stream ({NUM_BANDS}-band Wavelet EQ). Raise sliders to hear noise.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")


Starting Silent Stream (64-band Wavelet EQ). Raise sliders to hear noise.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# Initialize gains to 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        nodes = wp.get_level(6, order='freq')
        
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        processed_signal = wp.reconstruct(update=False)
        
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    # Show 2 decimal places for higher resolution
    label_widget.value = f"{new_val:.2f}"

for i in range(NUM_BANDS):
    lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # --- CHANGE: Taller sliders (400px) and finer step (0.01) ---
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    
    slider.band_index = i
    slider.label_ref = lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([lbl, slider], layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting High-Res Silent Stream. Raise sliders to hear noise.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting High-Res Silent Stream. Raise sliders to hear noise.


In [1]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# Initialize gains to 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        nodes = wp.get_level(6, order='freq')
        
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        processed_signal = wp.reconstruct(update=False)
        
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

# Calculate bandwidth per band
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.2f}"

for i in range(NUM_BANDS):
    # Top Label (Gain)
    gain_lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # Slider (Long & Fine)
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    
    # Bottom Label (Frequency)
    # Calculate start freq of this band
    freq_start = i * bandwidth
    
    # Format: 1000+ uses "k" (e.g., 1.2k), otherwise just Hz
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(
        value=freq_text,
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    # Linkage
    slider.band_index = i
    slider.label_ref = gain_lbl
    slider.observe(update_gain, names='value')
    
    # Stack: Gain -> Slider -> Freq
    col = widgets.VBox(
        [gain_lbl, slider, freq_lbl], 
        layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px')
    )
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Control Buttons ---
stop_btn = widgets.Button(
    description='Stop Noise',
    button_style='danger',
    icon='stop'
)

def stop_stream(b):
    global global_stream
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()
        print("Noise Stopped.")
    stop_btn.disabled = True

stop_btn.on_click(stop_stream)

# --- 4. Start Execution ---
print(f"Starting Stream. Bandwidth per slider: ~{int(bandwidth)}Hz")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([stop_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Stream. Bandwidth per slider: ~344Hz


In [2]:
import numpy as np
import sounddevice as sd
import ipywidgets as widgets
import pywt
from IPython.display import display

# --- Configuration ---
SAMPLE_RATE = 44100
BLOCK_SIZE = 4096
NUM_BANDS = 64
WAVELET_NAME = 'db1'
NOISE_VOLUME = 0.2

global_stream = None 

# Initialize gains to 0.0
gains = np.zeros(NUM_BANDS, dtype=np.float32)

# Global Mute Flag
is_muted = False

# --- 1. The Audio Callback ---
def audio_callback(outdata, frames, time, status):
    if status:
        print(status)
    
    # Check if muted
    if is_muted:
        outdata.fill(0)
        return

    # Generate White Noise
    noise = np.random.uniform(-1, 1, size=frames) * NOISE_VOLUME
    
    try:
        # Wavelet Decomposition
        wp = pywt.WaveletPacket(data=noise, wavelet=WAVELET_NAME, mode='symmetric', maxlevel=6)
        
        # Get leaf nodes (subbands) ordered by frequency
        nodes = wp.get_level(6, order='freq')
        
        # Apply Gains
        count = min(len(nodes), len(gains))
        for i in range(count):
            nodes[i].data = nodes[i].data * gains[i]
            
        # Reconstruct Signal
        processed_signal = wp.reconstruct(update=False)
        
        # Safety Clipping & Buffer Filling
        if len(processed_signal) > frames:
            processed_signal = processed_signal[:frames]
        elif len(processed_signal) < frames:
            processed_signal = np.pad(processed_signal, (0, frames - len(processed_signal)))
            
        outdata[:, 0] = processed_signal
        
    except Exception as e:
        print(f"Callback Error: {e}")
        outdata.fill(0)

# --- 2. The UI Logic ---
slider_containers = []

# Calculate bandwidth per band
nyquist = SAMPLE_RATE / 2
bandwidth = nyquist / NUM_BANDS

def update_gain(change):
    new_val = change['new']
    band_index = change['owner'].band_index
    label_widget = change['owner'].label_ref
    gains[band_index] = new_val
    label_widget.value = f"{new_val:.2f}"

for i in range(NUM_BANDS):
    # Top Label (Gain)
    gain_lbl = widgets.Label(
        value="0.00",
        layout=widgets.Layout(width='30px', justify_content='center')
    )
    
    # Slider
    slider = widgets.FloatSlider(
        value=0.0, min=0.0, max=5.0, step=0.01,
        orientation='vertical', readout=False,
        layout=widgets.Layout(width='20px', height='400px') 
    )
    
    # Bottom Label (Frequency)
    freq_start = i * bandwidth
    if freq_start >= 1000:
        freq_text = f"{freq_start/1000:.1f}k"
    else:
        freq_text = f"{int(freq_start)}"

    freq_lbl = widgets.Label(
        value=freq_text,
        layout=widgets.Layout(width='30px', justify_content='center', margin='5px 0px 0px 0px')
    )
    
    slider.band_index = i
    slider.label_ref = gain_lbl
    slider.observe(update_gain, names='value')
    
    col = widgets.VBox([gain_lbl, slider, freq_lbl], 
                       layout=widgets.Layout(align_items='center', margin='0px 2px 0px 0px'))
    slider_containers.append(col)

eq_box = widgets.HBox(slider_containers, layout=widgets.Layout(overflow='x scroll', width='100%'))

# --- 3. Mute Button Logic ---
mute_btn = widgets.Button(
    description='Mute',
    button_style='danger', # Red initially
    icon='volume-off'
)

def toggle_mute(b):
    global is_muted
    is_muted = not is_muted
    
    if is_muted:
        mute_btn.description = "Unmute"
        mute_btn.button_style = 'success' # Green
        mute_btn.icon = 'volume-up'
    else:
        mute_btn.description = "Mute"
        mute_btn.button_style = 'danger' # Red
        mute_btn.icon = 'volume-off'

mute_btn.on_click(toggle_mute)

# --- 4. Start Execution ---
print(f"Starting Stream. Use the button below to Mute/Unmute.")
try:
    if global_stream is not None:
        global_stream.stop()
        global_stream.close()

    global_stream = sd.OutputStream(samplerate=SAMPLE_RATE,
                                    blocksize=BLOCK_SIZE,
                                    channels=1,
                                    callback=audio_callback)
    global_stream.start()
    
    display(widgets.VBox([mute_btn, eq_box]))
    
except Exception as e:
    print(f"Error starting stream: {e}")

Starting Stream. Use the button below to Mute/Unmute.
